In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import xgboost as xgb
# from sklearn.model_selection import train_test_split

In [2]:
pd.set_option('display.max_columns', 10)
# pd.set_option('display.max_rows', None) 

In [4]:
df = pd.read_csv('assignment_2_train.csv')
df.head(5)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,...,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,...,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,...,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,...,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,...,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,...,0.0,0.0,0.0,0.0,0.0


In [4]:
df.shape

(180000, 394)

In [5]:
df.describe()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,...,V335,V336,V337,V338,V339
count,1.800000e+05,180000.000000,1.800000e+05,180000.000000,180000.000000,...,47996.000000,47996.000000,47996.000000,47996.000000,47996.000000
mean,3.077000e+06,0.028561,1.909818e+06,129.514237,9860.225806,...,18.404676,12.437524,26.798593,32.005349,28.492421
std,5.196167e+04,0.166570,1.039029e+06,206.515032,4910.778205,...,120.896921,99.305885,252.695682,284.870583,258.372941
min,2.987000e+06,0.000000,8.640000e+04,0.292000,1001.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.032000e+06,0.000000,1.091681e+06,44.000000,6019.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.077000e+06,0.000000,1.884075e+06,75.000000,9633.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.121999e+06,0.000000,2.693196e+06,125.000000,14182.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.166999e+06,1.000000,3.958317e+06,5278.950000,18396.000000,...,3350.000000,3250.000000,8250.000000,8250.000000,8250.000000


In [5]:
df_train = df[:round(df.shape[0]*0.8)].copy()
df_valid = df[round(df.shape[0]*0.8):df.shape[0]].copy()

In [18]:
df_train_category = df[:round(df.shape[0]*0.8)].copy()
df_valid_category = df[round(df.shape[0]*0.8):df.shape[0]].copy()

In [64]:
print(df_train.shape, df_valid.shape)

(144000, 394) (36000, 394)


In [6]:
object_features = df_train.select_dtypes(include=[np.object])
print(f"count of object_features {object_features.shape[1]}")

object_features_col = list(object_features.columns)
object_features_col

count of object_features 14


['ProductCD',
 'card4',
 'card6',
 'P_emaildomain',
 'R_emaildomain',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9']

In [6]:
df_train[object_features_col].head(5)

,ProductCD,card4,card6,P_emaildomain,R_emaildomain,...,M5,M6,M7,M8,M9
0,W,discover,credit,NaN,NaN,...,F,T,NaN,NaN,NaN
1,W,mastercard,credit,gmail.com,NaN,...,T,T,NaN,NaN,NaN
2,W,visa,debit,outlook.com,NaN,...,F,F,F,F,F
3,W,mastercard,debit,yahoo.com,NaN,...,T,F,NaN,NaN,NaN
4,H,mastercard,credit,gmail.com,NaN,...,NaN,NaN,NaN,NaN,NaN


In [7]:
numerical_features = df_train.select_dtypes(include=[np.number])
print(f"count of numeric_features {numerical_features.shape[1]}")

numerical_features.columns

count of numeric_features 380


Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'card1',
       'card2', 'card3', 'card5', 'addr1', 'addr2',
       ...
       'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
       'V339'],
      dtype='object', length=380)

In [8]:
continuous_feature = [
    feature for feature in numerical_features
    if feature not in ["TransactionID", "isFraud"]]

print(f"Continuous Feature Count {len(continuous_feature)}")

Continuous Feature Count 378


In [9]:
obj_and_dig = continuous_feature + object_features_col

In [10]:
target = 'isFraud'

In [17]:
df_train[continuous_feature].describe()

,TransactionDT,TransactionAmt,card1,card2,card3,...,V335,V336,V337,V338,V339
count,1.440000e+05,144000.000000,144000.000000,142057.000000,143997.000000,...,45081.000000,45081.000000,45081.000000,45081.000000,45081.000000
mean,1.529852e+06,127.722069,9867.081715,370.664888,153.539782,...,17.764731,12.464351,27.619025,32.538314,29.031416
std,7.786533e+05,199.985644,4923.876088,159.151352,11.607701,...,121.750867,101.047370,260.004327,292.002110,264.609907
min,8.640000e+04,0.292000,1001.000000,100.000000,100.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.721920e+05,44.075000,6019.000000,215.000000,150.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.615718e+06,75.000000,9633.000000,385.000000,150.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.139948e+06,126.000000,14276.000000,514.000000,150.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.956469e+06,5094.950000,18396.000000,600.000000,231.000000,...,3350.000000,3250.000000,8250.000000,8250.000000,8250.000000


In [11]:
for feature in object_features:
    namber_val = 0
    for unic_val in df_train[feature].unique():
        namber_val += 1
        df_train.loc[df_train[feature] == unic_val, feature] = namber_val
        df_valid.loc[df_valid[feature] == unic_val, feature] = namber_val
    df_train[feature] = pd.to_numeric(df_train[feature])
    df_valid[feature] = pd.to_numeric(df_valid[feature])

## XGBoost 

In [14]:
params = {
    "booster": "gbtree",
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "learning_rate": 0.1,
    "n_estimators": 100,
    "reg_lambda": 100,
    "max_depth": 10,
    "gamma": 10,
    "nthread": 6,
    "seed": 27
}

In [38]:
dtrain = xgb.DMatrix(
    data=df_train[continuous_feature], label=df_train[target]
)
dvalid = xgb.DMatrix(
    data=df_valid[continuous_feature], label=df_valid[target]
)

In [42]:
model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=100,
    early_stopping_rounds=20,
    evals=[(dtrain, "train"), (dvalid, "valid")],
    verbose_eval=10,
    maximize=True,
)

[0]	train-auc:0.64131	valid-auc:0.63913
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 20 rounds.
[10]	train-auc:0.79263	valid-auc:0.79753
[20]	train-auc:0.83832	valid-auc:0.83801
[30]	train-auc:0.87529	valid-auc:0.85721
[40]	train-auc:0.88696	valid-auc:0.86088
[50]	train-auc:0.89655	valid-auc:0.87270
[60]	train-auc:0.90250	valid-auc:0.87724
[70]	train-auc:0.90754	valid-auc:0.88091
[80]	train-auc:0.91092	valid-auc:0.88334
[90]	train-auc:0.91360	valid-auc:0.88413
[99]	train-auc:0.91612	valid-auc:0.88703


In [23]:
dtrain2 = xgb.DMatrix(
    data=df_train[obj_and_dig], label=df_train[target]
)
dvalid2 = xgb.DMatrix(
    data=df_valid[obj_and_dig], label=df_valid[target]
)

In [24]:
model2 = xgb.train(
    params=params,
    dtrain=dtrain2,
    num_boost_round=100,
    early_stopping_rounds=20,
    evals=[(dtrain2, "train"), (dvalid2, "valid")],
    verbose_eval=10,
    maximize=True,
)

[0]	train-auc:0.64131	valid-auc:0.63913
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 20 rounds.
[10]	train-auc:0.79264	valid-auc:0.79753
[20]	train-auc:0.86040	valid-auc:0.84586
[30]	train-auc:0.87652	valid-auc:0.85310
[40]	train-auc:0.89047	valid-auc:0.86075
[50]	train-auc:0.90219	valid-auc:0.86481
[60]	train-auc:0.91023	valid-auc:0.87490
[70]	train-auc:0.91608	valid-auc:0.88086
[80]	train-auc:0.91976	valid-auc:0.88455
[90]	train-auc:0.92274	valid-auc:0.88649
[99]	train-auc:0.92503	valid-auc:0.88810


## LightGBM

In [12]:
import lightgbm as lgb

In [13]:
params_lgb = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.1,
    "n_estimators": 100,
    "n_jobs": 2,
    "seed": 27
}

In [14]:
dtrain_lgb = lgb.Dataset(
    data=df_train[continuous_feature], label=df_train[target]
)
dvalid_lgb = lgb.Dataset(
    data=df_valid[continuous_feature], label=df_valid[target]
)

In [15]:
model_lgb = lgb.train(
    params=params_lgb,
    train_set=dtrain_lgb,
    num_boost_round=100,
    valid_sets=[dtrain_lgb, dvalid_lgb],
    early_stopping_rounds=20,
    verbose_eval=10
)

/home/aleksandr/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.892023	valid_1's auc: 0.869761
[20]	training's auc: 0.909402	valid_1's auc: 0.880656
[30]	training's auc: 0.92492	valid_1's auc: 0.886609
[40]	training's auc: 0.938239	valid_1's auc: 0.894222
[50]	training's auc: 0.946974	valid_1's auc: 0.898087
[60]	training's auc: 0.952641	valid_1's auc: 0.899962
[70]	training's auc: 0.957052	valid_1's auc: 0.902165
[80]	training's auc: 0.96245	valid_1's auc: 0.903298
[90]	training's auc: 0.966559	valid_1's auc: 0.904594
[100]	training's auc: 0.970139	valid_1's auc: 0.905538
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.970139	valid_1's auc: 0.905538


In [36]:
dtrain_lgb = lgb.Dataset(
    data=df_train[obj_and_dig], label=df_train[target]
)
dvalid_lgb = lgb.Dataset(
    data=df_valid[obj_and_dig], label=df_valid[target]
)

In [17]:
model_lgb2 = lgb.train(
    params=params_lgb,
    train_set=dtrain_lgb,
    num_boost_round=100,
    valid_sets=[dtrain_lgb, dvalid_lgb],
    early_stopping_rounds=20,
    verbose_eval=10
)

Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.89663	valid_1's auc: 0.863318
[20]	training's auc: 0.917335	valid_1's auc: 0.874342
[30]	training's auc: 0.932657	valid_1's auc: 0.88574
[40]	training's auc: 0.944797	valid_1's auc: 0.894003
[50]	training's auc: 0.953274	valid_1's auc: 0.898292
[60]	training's auc: 0.959292	valid_1's auc: 0.904721
[70]	training's auc: 0.965812	valid_1's auc: 0.906735
[80]	training's auc: 0.970435	valid_1's auc: 0.907959
[90]	training's auc: 0.97416	valid_1's auc: 0.90762
[100]	training's auc: 0.976796	valid_1's auc: 0.909656
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.976796	valid_1's auc: 0.909656


In [15]:
model_lgb3 = lgb.train(
    params=params_lgb,
    train_set=dtrain_lgb,
    num_boost_round=100,
    categorical_feature="auto",
    valid_sets=[dtrain_lgb, dvalid_lgb],
    early_stopping_rounds=20,
    verbose_eval=10,
)

/home/aleksandr/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.892023	valid_1's auc: 0.869761
[20]	training's auc: 0.909402	valid_1's auc: 0.880656
[30]	training's auc: 0.92492	valid_1's auc: 0.886609
[40]	training's auc: 0.938239	valid_1's auc: 0.894222
[50]	training's auc: 0.946974	valid_1's auc: 0.898087
[60]	training's auc: 0.952641	valid_1's auc: 0.899962
[70]	training's auc: 0.957052	valid_1's auc: 0.902165
[80]	training's auc: 0.96245	valid_1's auc: 0.903298
[90]	training's auc: 0.966559	valid_1's auc: 0.904594
[100]	training's auc: 0.970139	valid_1's auc: 0.905538
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.970139	valid_1's auc: 0.905538


## CatBoost

In [16]:
import catboost as cb

In [17]:
cb_params = {
    "n_estimators": 100,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 10,
    "max_depth": 10,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 20,
    "thread_count": 2,
    "random_seed": 27
}

In [20]:
model_cb = cb.CatBoostClassifier(**cb_params)
model_cb.fit(df_train[continuous_feature], df_train[target],
             eval_set=[(df_train[continuous_feature], df_train[target]),
                       (df_valid[continuous_feature], df_valid[target])
                      ]
            )

0:	test: 0.6485616	test1: 0.5794154	best: 0.5794154 (0)	total: 1.35s	remaining: 2m 14s
10:	test: 0.8227130	test1: 0.7959778	best: 0.7959778 (10)	total: 13.6s	remaining: 1m 49s
20:	test: 0.8370829	test1: 0.8183242	best: 0.8184439 (18)	total: 25.9s	remaining: 1m 37s
30:	test: 0.8467320	test1: 0.8226845	best: 0.8226845 (30)	total: 38.8s	remaining: 1m 26s
40:	test: 0.8550070	test1: 0.8333528	best: 0.8344817 (39)	total: 51.3s	remaining: 1m 13s
50:	test: 0.8620891	test1: 0.8403463	best: 0.8417666 (48)	total: 1m 2s	remaining: 1m
60:	test: 0.8647337	test1: 0.8435624	best: 0.8435624 (60)	total: 1m 15s	remaining: 48.2s
70:	test: 0.8669438	test1: 0.8452529	best: 0.8455735 (68)	total: 1m 27s	remaining: 35.9s
80:	test: 0.8682912	test1: 0.8448355	best: 0.8456940 (72)	total: 1m 40s	remaining: 23.6s
90:	test: 0.8694562	test1: 0.8467578	best: 0.8467578 (90)	total: 1m 53s	remaining: 11.2s
99:	test: 0.8706538	test1: 0.8474504	best: 0.8478128 (95)	total: 2m 5s	remaining: 0us

bestTest = 0.8478127617
bestI

In [21]:
model_cb2 = cb.CatBoostClassifier(**cb_params)
model_cb2.fit(df_train[obj_and_dig], df_train[target],
             eval_set=[(df_train[obj_and_dig], df_train[target]),
                       (df_valid[obj_and_dig], df_valid[target])
                      ]
            )

0:	test: 0.6639600	test1: 0.5775589	best: 0.5775589 (0)	total: 1.24s	remaining: 2m 2s
10:	test: 0.8012865	test1: 0.7751651	best: 0.7751651 (10)	total: 13.4s	remaining: 1m 48s
20:	test: 0.8342819	test1: 0.8037390	best: 0.8061733 (19)	total: 25.8s	remaining: 1m 37s
30:	test: 0.8462875	test1: 0.8239714	best: 0.8239714 (30)	total: 38.4s	remaining: 1m 25s
40:	test: 0.8545706	test1: 0.8338340	best: 0.8359394 (38)	total: 50.9s	remaining: 1m 13s
50:	test: 0.8644419	test1: 0.8426839	best: 0.8426839 (50)	total: 1m 3s	remaining: 1m
60:	test: 0.8692805	test1: 0.8482810	best: 0.8482810 (60)	total: 1m 15s	remaining: 48s
70:	test: 0.8739599	test1: 0.8511378	best: 0.8511378 (70)	total: 1m 27s	remaining: 35.6s
80:	test: 0.8762780	test1: 0.8530291	best: 0.8530291 (80)	total: 1m 39s	remaining: 23.3s
90:	test: 0.8795741	test1: 0.8537470	best: 0.8537470 (90)	total: 1m 50s	remaining: 11s
99:	test: 0.8813956	test1: 0.8548694	best: 0.8548694 (99)	total: 2m 2s	remaining: 0us

bestTest = 0.8548693945
bestIterat

In [23]:
df_train_category[object_features_col] = df_train_category[object_features_col].fillna('space')
df_valid_category[object_features_col] = df_valid_category[object_features_col].fillna('space')

In [24]:
model_cb3 = cb.CatBoostClassifier(**cb_params)
model_cb3.fit(df_train_category[obj_and_dig], df_train_category[target]
              ,eval_set=[(df_train_category[obj_and_dig], df_train_category[target])
                        ,(df_valid_category[obj_and_dig], df_valid_category[target])
                      ]
              ,cat_features=object_features_col
            )

0:	test: 0.6801172	test1: 0.6881362	best: 0.6881362 (0)	total: 3.32s	remaining: 5m 28s
10:	test: 0.8096569	test1: 0.8074129	best: 0.8088343 (8)	total: 19.9s	remaining: 2m 40s
20:	test: 0.8256781	test1: 0.8070352	best: 0.8091926 (14)	total: 36.7s	remaining: 2m 18s
30:	test: 0.8431389	test1: 0.8219585	best: 0.8219585 (30)	total: 53s	remaining: 1m 57s
40:	test: 0.8529222	test1: 0.8336821	best: 0.8358722 (38)	total: 1m 8s	remaining: 1m 38s
50:	test: 0.8598114	test1: 0.8422745	best: 0.8422745 (50)	total: 1m 23s	remaining: 1m 20s
60:	test: 0.8635678	test1: 0.8462138	best: 0.8465213 (59)	total: 1m 39s	remaining: 1m 3s
70:	test: 0.8658335	test1: 0.8472609	best: 0.8472609 (70)	total: 1m 56s	remaining: 47.5s
80:	test: 0.8688677	test1: 0.8481864	best: 0.8487551 (77)	total: 2m 11s	remaining: 30.9s
90:	test: 0.8723565	test1: 0.8498490	best: 0.8499594 (89)	total: 2m 26s	remaining: 14.5s
99:	test: 0.8758774	test1: 0.8519658	best: 0.8519658 (99)	total: 2m 39s	remaining: 0us

bestTest = 0.8519657579
be